In [188]:
import credentials as cd
from fyers_apiv3 import fyersModel
import pandas as pd
import datetime as dt
import ta

In [189]:
with open("access.txt", "r") as a:
    access_token = a.read()
client_id = cd.client_id

# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(
    client_id=client_id, is_async=False, token=access_token, log_path=""
)

In [190]:
# def gethistory(symbol1, type, range_from, range_to):
#     symbol = "NSE:" + symbol1 + "-" + type
#     start = pd.to_datetime(range_from)
#     end = pd.to_datetime(range_to)
#     sdata = pd.DataFrame()
#     while start <= end:
#         end2 = start + pd.DateOffset(days=60)
#         data = {
#             "symbol": symbol,
#             "resolution": "1",
#             "date_format": "1",
#             "range_from": start.strftime("%Y-%m-%d"),
#             "range_to": end2.strftime("%Y-%m-%d"),
#             "cont_flag": "1",
#         }
#         s = fyers.history(data)
#         s = pd.DataFrame(s["candles"])
#         sdata = pd.concat([sdata, s], ignore_index=True)
#         start = end2 + dt.timedelta(1)
#     sdata.columns = ["date", "open", "high", "low", "close", "volume"]
#     sdata["date"] = pd.to_datetime(sdata["date"], unit="s")
#     sdata.date = sdata.date.dt.tz_localize("UTC").dt.tz_convert("Asia/Kolkata")
#     sdata["date"] = sdata["date"].dt.tz_localize(None)
#     sdata = sdata.set_index("date")
#     return sdata


# data = gethistory("NIFTY50", "INDEX", "2024-03-01", "2024-05-09")
# print(data)
# data.to_csv("NIFTY50.csv")

In [191]:
# Collecting the Symbol or Index Value from the Data Receiver....
def gethistory(symbol1, type, range_from, range_to):
    symbol = "NSE:" + symbol1 + "-" + type
    start = pd.to_datetime(range_from)
    end = pd.to_datetime(range_to)
    sdata = pd.DataFrame()
    while start <= end:
        end2 = start + pd.DateOffset(days=60)
        data = {
            "symbol": symbol,
            "resolution": "1",
            "date_format": "1",
            "range_from": start.strftime("%Y-%m-%d"),
            "range_to": end2.strftime("%Y-%m-%d"),
            "cont_flag": "1",
        }
        s = fyers.history(data)
        s = pd.DataFrame(s["candles"])
        sdata = pd.concat([sdata, s], ignore_index=True)
        start = end2 + dt.timedelta(1)
    sdata.columns = ["date", "open", "high", "low", "close", "volume"]
    sdata["date"] = pd.to_datetime(sdata["date"], unit="s")
    sdata.date = sdata.date.dt.tz_localize("UTC").dt.tz_convert("Asia/Kolkata")
    sdata["date"] = sdata["date"].dt.tz_localize(None)
    sdata = sdata.set_index("date")
    return sdata


data = gethistory("NIFTY50", "INDEX", "2024-03-01", "2024-05-09")
print(data)

# Technical Analysis
# Calculate the 9-period and 13-period EMA of the closing prices
data["EMA9"] = ta.trend.ema_indicator(data.close, window=9)
data["EMA13"] = ta.trend.ema_indicator(data.close, window=13)

# Calculate the MACD with fast period of 9, slow period of 13, and signal period of 9
macd = ta.trend.MACD(data.close, window_fast=9, window_slow=13, window_sign=9)

data["MACD"], data["MACD_signal"], data["MACD_hist"] = (
    macd.macd(),
    macd.macd_signal(),
    macd.macd_diff(),
)


# Initialize an empty list to store the signals
signals = []
# Loop over all rows in the data (starting from the second one)
for i in range(
    1, len(data) - 1
):  # Adjusted to len(data)-1 to avoid index error in the next line
    # Get the current row, the previous row, and the next row
    current_row = data.iloc[i]
    previous_row = data.iloc[i - 1]
    next_row = data.iloc[i + 1]  # Added next_row

    # Identify the trend based on the MACD and Signal Line
    if current_row["MACD"] > current_row["MACD_signal"]:
        trend = "up"
    else:
        trend = "down"

    # Check for EMA crossovers

    if (
        previous_row["EMA9"] > previous_row["EMA13"]
        and current_row["EMA9"] < current_row["EMA13"]
        and trend == "down"
    ):

        signals.append(
            {
                "timestamp": current_row.name,
                "price": current_row["close"],
                "signal": "sell",
                "MACD": current_row["MACD"],
                "Signal_Line": current_row["MACD_signal"],
                "trend": trend,
            }
        )

        # If MACD provides an Down Trend, then PE Option Stike Price to Buy
        def format_date(date):
            year = date.year % 100  # get last two digits of year
            month = str(date.month).zfill(1)  # remove leading zero from month
            day = str(date.day).zfill(2)  # keep leading zero for day
            return f"{year}{month}{day}"

        op_data = {"symbol": "NSE:NIFTY50-INDEX", "strikecount": 1, "timestamp": ""}
        oc = fyers.optionchain(data=op_data)
        expiry = oc["data"]["expiryData"][0]["date"]
        expiry = format_date(pd.to_datetime(expiry))  # reformat the date
        sp = round(data.iloc[-1]["close"] / 50) * 50
        pstrike = "NSE:NIFTY" + expiry + str(sp) + "PE"
        print(pstrike)

        strike_data = {
            "symbol": pstrike,
            "resolution": "1",
            "date_format": "1",
            "range_from": "2024-03-01",
            "range_to": "2024-05-09",
            "cont_flag": "1",
        }

        p_response = fyers.history(data=strike_data)
        p_df = p_response["candles"]
        strike = pd.DataFrame(p_df)

        strike.columns = ["date", "open", "high", "low", "close", "volume"]
        strike["date"] = pd.to_datetime(strike["date"], unit="s")
        strike["date"] = (
            strike["date"].dt.tz_localize("utc").dt.tz_convert("Asia/Kolkata")
        )
        strike["date"] = strike["date"].dt.tz_localize(None)
        strike = strike.set_index("date")

        # print(strike)
        # print(op_data)
        # print(oc)
        # print(expiry)
        # print(sp)
        # print(strike_data)
        print(p_response)

        data = strike
        # Calculate the 9-period and 13-period EMA of the closing prices
        data["EMA9"] = ta.trend.ema_indicator(data.close, window=9)
        data["EMA13"] = ta.trend.ema_indicator(data.close, window=13)

        # Calculate the MACD with fast period of 9, slow period of 13, and signal period of 9
        macd = ta.trend.MACD(data.close, window_fast=9, window_slow=13, window_sign=9)
        data["MACD"], data["MACD_signal"], data["MACD_hist"] = (
            macd.macd(),
            macd.macd_signal(),
            macd.macd_diff(),
        )

        # Initialize an empty list to store the signals
        signals = []

        # Loop over all rows in the data (starting from the second one)
        for i in range(1, len(data) - 1):
            # Get the current row, the previous row, and the next row
            current_row = data.iloc[i]
            previous_row = data.iloc[i - 1]

            # Identify the trend based on the MACD and Signal Line
            if current_row["MACD"] > current_row["MACD_signal"]:
                trend = "up"
            else:
                trend = "down"

            # Check for EMA crossovers

            if (
                previous_row["EMA9"] < previous_row["EMA13"]
                and current_row["EMA9"] > current_row["EMA13"]
                and trend == "up"
            ):
                signals.append(
                    {
                        "timestamp": current_row.name,
                        "price": current_row["close"],
                        "signal": "buy",
                        "MACD": current_row["MACD"],
                        "Signal_Line": current_row["MACD_signal"],
                        "trend": trend,
                        "stop_loss": current_row["EMA9"],
                        "target": current_row["close"]
                        + 20,  # target is 20 points above the entry price
                    }
                )

    elif (
        previous_row["EMA9"] < previous_row["EMA13"]
        and current_row["EMA9"] > current_row["EMA13"]
        and trend == "up"
    ):

        signals.append(
            {
                "timestamp": current_row.name,
                "price": current_row["close"],
                "signal": "buy",
                "MACD": current_row["MACD"],
                "Signal_Line": current_row["MACD_signal"],
                "trend": trend,
            }
        )

        def format_date(date):
            year = date.year % 100  # get last two digits of year
            month = str(date.month).zfill(1)  # remove leading zero from month
            day = str(date.day).zfill(2)  # keep leading zero for day
            return f"{year}{month}{day}"

        op_data = {"symbol": "NSE:NIFTY50-INDEX", "strikecount": 1, "timestamp": ""}
        oc = fyers.optionchain(data=op_data)
        expiry = oc["data"]["expiryData"][0]["date"]
        expiry = format_date(pd.to_datetime(expiry))  # reformat the date
        sp = round(data.iloc[-1]["price"] / 50) * 50
        cstrike = "NSE:NIFTY" + expiry + str(sp) + "CE"
        print(cstrike)

        strike_data = {
            "symbol": cstrike,
            "resolution": "1",
            "date_format": "1",
            "range_from": "2024-03-01",
            "range_to": "2024-05-09",
            "cont_flag": "1",
        }

        response = fyers.history(data=strike_data)
        c_df = response["candles"]
        strike = pd.DataFrame(c_df)

        strike.columns = ["date", "open", "high", "low", "close", "volume"]
        strike["date"] = pd.to_datetime(strike["date"], unit="s")
        strike["date"] = (
            strike["date"].dt.tz_localize("utc").dt.tz_convert("Asia/Kolkata")
        )
        strike["date"] = strike["date"].dt.tz_localize(None)
        strike = strike.set_index("date")

        # print(strike)

        data = strike
        # Calculate the 9-period and 13-period EMA of the closing prices
        data["EMA9"] = ta.trend.ema_indicator(data.close, window=9)
        data["EMA13"] = ta.trend.ema_indicator(data.close, window=13)

        # Calculate the MACD with fast period of 9, slow period of 13, and signal period of 9
        macd = ta.trend.MACD(data.close, window_fast=9, window_slow=13, window_sign=9)
        data["MACD"], data["MACD_signal"], data["MACD_hist"] = (
            macd.macd(),
            macd.macd_signal(),
            macd.macd_diff(),
        )

        # Initialize an empty list to store the signals
        signals = []

        # Loop over all rows in the data (starting from the second one)
        for i in range(1, len(data) - 1):
            # Get the current row, the previous row, and the next row
            current_row = data.iloc[i]
            previous_row = data.iloc[i - 1]

            # Identify the trend based on the MACD and Signal Line
            if current_row["MACD"] > current_row["MACD_signal"]:
                trend = "up"
            else:
                trend = "down"

            # Check for EMA crossovers

            if (
                previous_row["EMA9"] < previous_row["EMA13"]
                and current_row["EMA9"] > current_row["EMA13"]
                and trend == "up"
            ):
                signals.append(
                    {
                        "timestamp": current_row.name,
                        "price": current_row["close"],
                        "signal": "buy",
                        "MACD": current_row["MACD"],
                        "Signal_Line": current_row["MACD_signal"],
                        "trend": trend,
                        "stop_loss": current_row["EMA9"],
                        "target": current_row["close"]
                        + 20,  # target is 20 points above the entry price
                    }
                )



# Convert the list of signals to a DataFrame for easier manipulation


signals_data = pd.DataFrame(signals)



# Calculate profit/loss



signals_data["profit_loss"] = signals_data["price"].diff()



# For 'buy' signals, we need to take the negative of the profit/loss


# Output the DataFrame to a CSV file


print(signals_data)

                         open      high       low     close  volume
date                                                               
2024-03-01 09:15:00  22048.30  22125.65  22047.75  22117.25       0
2024-03-01 09:16:00  22117.55  22124.05  22117.50  22118.40       0
2024-03-01 09:17:00  22119.45  22122.40  22114.90  22114.90       0
2024-03-01 09:18:00  22113.65  22122.65  22112.35  22121.75       0
2024-03-01 09:19:00  22122.20  22123.40  22114.10  22114.95       0
...                       ...       ...       ...       ...     ...
2024-05-10 15:25:00  22059.90  22060.95  22053.55  22053.55       0
2024-05-10 15:26:00  22053.05  22055.60  22050.40  22050.45       0
2024-05-10 15:27:00  22049.85  22056.45  22048.55  22052.15       0
2024-05-10 15:28:00  22051.85  22053.85  22041.40  22043.50       0
2024-05-10 15:29:00  22044.50  22054.90  22042.50  22054.00       0

[16980 rows x 5 columns]


C:\Users\Shiva Kumar G\AppData\Local\Temp\ipykernel_8312\3145037389.py:91: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  expiry = format_date(pd.to_datetime(expiry))  # reformat the date


NSE:NIFTY2451622050PE
{'candles': [[1713429420, 275.6, 275.6, 275.6, 275.6, 50], [1713429480, 275.6, 275.6, 275.6, 275.6, 0], [1713429540, 275.6, 275.6, 275.6, 275.6, 0], [1713429600, 275.6, 275.6, 275.6, 275.6, 0], [1713429660, 275.6, 275.6, 275.6, 275.6, 0], [1713429720, 275.6, 275.6, 275.6, 275.6, 0], [1713429780, 275.6, 275.6, 275.6, 275.6, 0], [1713429840, 275.6, 275.6, 275.6, 275.6, 0], [1713429900, 275.6, 275.6, 275.6, 275.6, 0], [1713429960, 275.6, 275.6, 275.6, 275.6, 0], [1713430020, 275.6, 275.6, 275.6, 275.6, 0], [1713430080, 275.6, 275.6, 275.6, 275.6, 0], [1713430140, 275.6, 275.6, 275.6, 275.6, 0], [1713430200, 275.6, 275.6, 275.6, 275.6, 0], [1713430260, 275.6, 275.6, 275.6, 275.6, 0], [1713430320, 275.6, 275.6, 275.6, 275.6, 0], [1713430380, 275.6, 275.6, 275.6, 275.6, 0], [1713430440, 275.6, 275.6, 275.6, 275.6, 0], [1713430500, 275.6, 275.6, 275.6, 275.6, 0], [1713430560, 275.6, 275.6, 275.6, 275.6, 0], [1713430620, 275.6, 275.6, 275.6, 275.6, 0], [1713430680, 275.6,

C:\Users\Shiva Kumar G\AppData\Local\Temp\ipykernel_8312\3145037389.py:91: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  expiry = format_date(pd.to_datetime(expiry))  # reformat the date


NSE:NIFTY24516200PE


KeyError: 'candles'

In [ ]:
print(df)

[[1713429420, 275.6, 275.6, 275.6, 275.6, 50], [1713429480, 275.6, 275.6, 275.6, 275.6, 0], [1713429540, 275.6, 275.6, 275.6, 275.6, 0], [1713429600, 275.6, 275.6, 275.6, 275.6, 0], [1713429660, 275.6, 275.6, 275.6, 275.6, 0], [1713429720, 275.6, 275.6, 275.6, 275.6, 0], [1713429780, 275.6, 275.6, 275.6, 275.6, 0], [1713429840, 275.6, 275.6, 275.6, 275.6, 0], [1713429900, 275.6, 275.6, 275.6, 275.6, 0], [1713429960, 275.6, 275.6, 275.6, 275.6, 0], [1713430020, 275.6, 275.6, 275.6, 275.6, 0], [1713430080, 275.6, 275.6, 275.6, 275.6, 0], [1713430140, 275.6, 275.6, 275.6, 275.6, 0], [1713430200, 275.6, 275.6, 275.6, 275.6, 0], [1713430260, 275.6, 275.6, 275.6, 275.6, 0], [1713430320, 275.6, 275.6, 275.6, 275.6, 0], [1713430380, 275.6, 275.6, 275.6, 275.6, 0], [1713430440, 275.6, 275.6, 275.6, 275.6, 0], [1713430500, 275.6, 275.6, 275.6, 275.6, 0], [1713430560, 275.6, 275.6, 275.6, 275.6, 0], [1713430620, 275.6, 275.6, 275.6, 275.6, 0], [1713430680, 275.6, 275.6, 275.6, 275.6, 0], [1713430

In [ ]:
# import ta


# def format_date(date):
#     year = date.year % 100  # get last two digits of year
#     month = str(date.month).zfill(1)  # remove leading zero from month
#     day = str(date.day).zfill(2)  # keep leading zero for day
#     return f"{year}{month}{day}"


# op_data = {"symbol": "NSE:NIFTY50-INDEX", "strikecount": 1, "timestamp": ""}
# response = fyers.optionchain(data=op_data)
# expiry = response["data"]["expiryData"][0]["date"]
# expiry = format_date(pd.to_datetime(expiry))  # reformat the date
# sp = round(signals_data.iloc[-1]["price"] / 50) * 50
# pstrike = "NSE:NIFTY" + expiry + str(sp) + "PE"
# cstrike = "NSE:NIFTY" + expiry + str(sp) + "CE"
# print(pstrike)
# print(cstrike)

# strike_data = {
#     "symbol": cstrike,
#     "resolution": "1",
#     "date_format": "1",
#     "range_from": "2024-03-01",
#     "range_to": "2024-05-09",
#     "cont_flag": "1",
# }

# response = fyers.history(data=strike_data)
# df = response["candles"]
# strike = pd.DataFrame(df)

# strike.columns = ["date", "open", "high", "low", "close", "volume"]
# strike["date"] = pd.to_datetime(strike["date"], unit="s")
# strike["date"] = strike["date"].dt.tz_localize("utc").dt.tz_convert("Asia/Kolkata")
# strike["date"] = strike["date"].dt.tz_localize(None)
# strike = strike.set_index("date")

# # print(strike)


# data = strike
# # Calculate the 9-period and 13-period EMA of the closing prices
# data["EMA9"] = ta.trend.ema_indicator(data.close, window=9)
# data["EMA13"] = ta.trend.ema_indicator(data.close, window=13)

# # Calculate the MACD with fast period of 9, slow period of 13, and signal period of 9
# macd = ta.trend.MACD(data.close, window_fast=9, window_slow=13, window_sign=9)
# data["MACD"], data["MACD_signal"], data["MACD_hist"] = (
#     macd.macd(),
#     macd.macd_signal(),
#     macd.macd_diff(),
# )

# # Initialize an empty list to store the signals
# signals = []

# # Loop over all rows in the data (starting from the second one)
# for i in range(1, len(data) - 1):
#     # Get the current row, the previous row, and the next row
#     current_row = data.iloc[i]
#     previous_row = data.iloc[i - 1]

#     # Identify the trend based on the MACD and Signal Line
#     if current_row["MACD"] > current_row["MACD_signal"]:
#         trend = "up"
#     else:
#         trend = "down"

#     # Check for EMA crossovers
#     if (
#         previous_row["EMA9"] > previous_row["EMA13"]
#         and current_row["EMA9"] < current_row["EMA13"]
#         and trend == "down"
#     ):
#         signals.append(
#             {
#                 "timestamp": current_row.name,
#                 "price": current_row["close"],
#                 "signal": "sell",
#                 "MACD": current_row["MACD"],
#                 "Signal_Line": current_row["MACD_signal"],
#                 "trend": trend,
#                 "stop_loss": current_row["EMA9"],
#                 "target": current_row["close"]
#                 - 20,  # target is 20 points below the entry price
#             }
#         )
#     elif (
#         previous_row["EMA9"] < previous_row["EMA13"]
#         and current_row["EMA9"] > current_row["EMA13"]
#         and trend == "up"
#     ):
#         signals.append(
#             {
#                 "timestamp": current_row.name,
#                 "price": current_row["close"],
#                 "signal": "buy",
#                 "MACD": current_row["MACD"],
#                 "Signal_Line": current_row["MACD_signal"],
#                 "trend": trend,
#                 "stop_loss": current_row["EMA9"],
#                 "target": current_row["close"]
#                 + 20,  # target is 20 points above the entry price
#             }
#         )

# # Convert the list of signals to a DataFrame for easier manipulation
# signals_data = pd.DataFrame(signals)

# # Calculate profit/loss
# signals_data["profit_loss"] = signals_data["price"].diff()
# # For 'buy' signals, we need to take the negative of the profit/loss
# signals_data.loc[signals_data["signal"] == "buy", "profit_loss"] *= -1

# # Add additional columns
# signals_data["timestamp"] = signals_data["timestamp"]
# signals_data["index_price"] = signals_data["price"]
# signals_data["ATM_strike"] = sp  # Assuming 'sp' is the ATM strike
# signals_data["entry_price"] = signals_data["price"]
# signals_data["stoploss"] = signals_data["stop_loss"]
# signals_data["target"] = signals_data["target"]
# signals_data["exit"] = signals_data["price"].shift(
#     -1
# )  # Assuming the exit price is the price of the next signal
# signals_data["profit_and_loss"] = signals_data["exit"] - signals_data["entry_price"]
# signals_data["MACD_trend"] = signals_data["MACD"].apply(
#     lambda x: "up" if x > 0 else "down"
# )

# # Output the DataFrame to a CSV file
# signals_data.to_csv("optionSignal.csv", index=False)

# print(signals_data)